In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import statsmodels.api as sm
from scipy.optimize import minimize
import scipy as sc
import glob
import matplotlib.pyplot as plt
import os
#from arch import arch_model
#from fnGARCH_2 import *

In [ ]:
data1 = pd.read_csv"

In [5]:

def fnVaR_n(dfData, weights, alpha, lookback, percent):
    """
    purpose:
        calculates VaR and ES with variance-covariance method
        using a normal distribution
    input:
        dfData, dataframe
        weights, weights vector
        apha, significance value
        lookback, lookback period
        percent, True= express in percent
    output:
        VaR expressed in losses (positive value)
    """
    # check alpha
    if alpha > 0.5:
        alpha= 1-alpha
    
    # express years in days
    years= lookback
    # select last n years of data
    DataTemp= dfData.iloc[-years:,0:4]
    # calculate covariance
    cov= np.cov(DataTemp, rowvar= False)
    # calculate variance
    variance= weights@cov@weights.T
    variance= variance[0][0]
    # calculate mean
    mean= np.array(np.mean(DataTemp))@weights.T
    mean= mean[0]
    # calculate quantile
    z= sc.stats.norm.ppf(alpha)
    VaR= mean + z * np.sqrt(variance)
    VaR= -VaR
    
    # calculate ES
    ES= -VaR
    DataTemp_ES= dfData.iloc[-years:,6]
    DataTemp_ES= DataTemp_ES[DataTemp_ES < ES]
    ES= -np.mean(DataTemp_ES)
    
    if percent == False:
        VaR= VaR*dfData.iloc[-1,7]
        ES= ES*dfData.iloc[-1,7]
        ES= round(ES, 2)
        VaR= round(VaR, 2)
    
    return VaR, ES
